In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.container { hight:100% !important; }</style>"))

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
import csv
import numpy as np
from tqdm import tqdm
import pandas as pd

try:
    table_list = []
    conn = mysql.connector.connect(host='localhost', user='root',  
                        password='toor')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor(buffered=True)
        #cursor.execute("CREATE DATABASE case_test")
        cursor.execute("SHOW DATABASES")
        for (database) in cursor:
            if database[0] not in ["information_schema","performance_schema","mysql","sys"]:
                #print(database[0])
                database_case = database[0]
                sql_use = "USE " + database_case
        cursor.execute(sql_use)
        cursor.execute("SHOW TABLES")
        for table in cursor:
            table_list.append(table[0])
        print("Database connected: use database: " + database_case + ", tables: " + str(table_list))
except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
#注意：各类型记录应保持字段一致，如应都包含“是否删除”字段等，注意取证时的数据提取范围和设置

In [ ]:
create_table_case = """CREATE TABLE 案件信息 (
    案件号 varchar(255),
    嫌疑人微信号 varchar(255),
    嫌疑人姓名 varchar(255)
);"""

create_table_wechat_account = """CREATE TABLE 微信帐号信息 (
    案件号 varchar(255),
    嫌疑人身份证号 varchar(255),
    嫌疑人姓名 varchar(255),
    微信号 varchar(255),
    昵称 varchar(255),
    微信ID varchar(255),
    QQ号码 varchar(255),
    密钥 varchar(255),
    性别 varchar(255),
    个性签名 text,
    备注 varchar(255),
    省 varchar(255),
    市 varchar(255),
    头像 varchar(255),
    邮箱地址 varchar(255),
    登录时间 DATETIME,
    国家 varchar(255),
    手机号 varchar(255),
    真实名 varchar(255),
    用户UIN int
);"""

create_table_wechat_chat = """CREATE TABLE 好友聊天记录 (
    案件号 varchar(255),
    嫌疑人身份证号 varchar(255),
    嫌疑人姓名 varchar(255),
    发送者号码 varchar(255),
    发送者名称 varchar(255),
    发送者备注 varchar(255),
    接收者号码 varchar(255),
    接收者名称 varchar(255),
    接收者备注 varchar(255),
    发送时间 DATETIME,
    内容 TEXT,
    附件路径 varchar(255),
    类型 varchar(255),
    是否下载 varchar(255),
    是否已读 varchar(255),
    语音文本 varchar(255),
    内容翻译 varchar(255),
    是否删除 varchar(255)
);"""

create_table_app_geo = """CREATE TABLE 应用位置信息 (
    案件号 varchar(255),
    嫌疑人身份证号 varchar(255),
    嫌疑人姓名 varchar(255),
    应用名称 varchar(255),
    发送者账号 varchar(255),
    发送者昵称 varchar(255),
    纬度 varchar(255),
    经度 varchar(255),
    地点 varchar(255),
    详细地址 varchar(255),
    时间 DATETIME,
    来源账号ID varchar(255),
    来源账号昵称 varchar(255)
);"""
table_list = [create_table_case,create_table_wechat_account,create_table_wechat_chat,create_table_app_geo]

for table in table_list:
    try:
        cursor.execute(table)
    except Error as e:
        print("[INFO]: " + e.msg)
    

In [1]:
def data_import():
    print("---- 微信聊天记录数据录入与检索,录入中... -----")
    #case_csv_phone_path = "E://BaiduNetdiskDownload//案例//案例4//取证报告//287-J1_20211122151604_csv//V1924A//"
    
    case_csv_phone_path = input_data_import_text.value + "//"
    print("当前 csv 报告目录: " + case_csv_phone_path)
    dirs = os.listdir( case_csv_phone_path )
    x = y = z = g = 0
    for dir in dirs:
        x = x + 1
        df = pd.read_csv(case_csv_phone_path + dir, encoding='utf-16',index_col=False, sep='\t')
        insert_index_1 = 0
        insert_index_2 = 1
        insert_index_3 = 2
        insert_index_4 = 3
        insert_colname_1 = "案件号"
        insert_colname_2 = "嫌疑人身份证号"
        insert_colname_3 = "嫌疑人姓名"
        insert_value_1 = input_case_number_text.value
        insert_value_2 = input_suspect_wechat_account_text.value
        insert_value_3 = input_suspect_name_text.value
        df.insert(loc=insert_index_1, column=insert_colname_1, value=insert_value_1)
        df.insert(loc=insert_index_2, column=insert_colname_2, value=insert_value_2)
        df.insert(loc=insert_index_3, column=insert_colname_3, value=insert_value_3)
        try:
            if "位置信息" in dir:
                df['时间'].fillna(0, inplace=True)
                df.fillna("null", inplace=True)
                try:
                    for i,row in df.iterrows():
                        g = g + 1
                        #here %S means string values : 13 $s
                        sql = "INSERT INTO case_test.应用位置信息 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                        cursor.execute(sql, tuple(row))
                        conn.commit()
                        #print(cursor.rowcount, "record inserted.")
                except Error as e:
                    print("[Error]: " + dir + ', ' + e.msg)
            if "帐号信息" in dir:
                df['用户UIN'].fillna(0, inplace=True)
                df['登录时间'].fillna(0, inplace=True)
                df.fillna("null", inplace=True)
                try:
                    for i,row in df.iterrows():
                        #here %S means string values : 20 $s
                        z = z + 1
                        sql = "INSERT INTO case_test.微信帐号信息 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                        cursor.execute(sql, tuple(row))
                        conn.commit()
                        #print(cursor.rowcount, "record inserted.")
                except Error as e:
                    print("[Error]: " + dir + ', ' + e.msg)
            elif "聊天记录_好友聊天记录" in dir and ("公众号") not in dir:
                cols = { "column":[
                            {"col_name":"案件号","index":0},
                            {"col_name":"嫌疑人身份证号","index":1}, 
                            {"col_name":"嫌疑人姓名","index":2}, 
                            {"col_name":"发送者号码","index":3},
                            {"col_name":"发送者名称","index":4},
                            {"col_name":"发送者备注","index":5}, 
                            {"col_name":"接收者号码","index":6}, 
                            {"col_name":"接收者名称","index":7},
                            {"col_name":"接收者备注","index":8},
                            {"col_name":"发送时间","index":9}, 
                            {"col_name":"内容","index":10}, 
                            {"col_name":"附件路径","index":11},
                            {"col_name":"类型","index":12},
                            {"col_name":"是否下载","index":13}, 
                            {"col_name":"是否已读","index":14}, 
                            {"col_name":"语音文本","index":15},
                            {"col_name":"内容翻译","index":16},
                            {"col_name":"是否删除","index":17}
                          ]
                       }
                #print(df.columns)
                for item in cols['column']:
                    #print(item)
                    if item['col_name'] not in df.columns:
                        #print("missing " + item['col_name'])
                        df.insert(loc=item['index'], column=item['col_name'], value="null")
                df['发送时间'].fillna(0, inplace=True)
                df.fillna("null", inplace=True)

                try:
                    for i,row in df.iterrows():
                        y = y + 1
                        #here %S means string values : 18 $s
                        sql = "INSERT INTO case_test.好友聊天记录 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                        #print(tuple(row), len(tuple(row)))
                        #print(row)
                        cursor.execute(sql, tuple(row))
                        #print("Record inserted")
                        # the connection is not auto committed by default, so we must commit to save our changes
                        conn.commit()
                        #print(cursor.rowcount, "record inserted.")
                except Error as e:
                    print("[Error]: " + dir + ', ' + e.msg)
                    
            
        except FileNotFoundError as e:
            print("[Error]: ", e)
    print("== 录入完成, 扫描" + str(x) + "个 csv 文档，录入" + str(y) + "条聊天记录" + str(z) + "个微信帐号信息" + str(g) + "个应用位置记录 ==")
    
    
   

In [ ]:
def data_query_chat():
    wechat_account = input_query_wechat_account_text.value
    string = input_query_content_string_text.value
    case_number = input_query_case_number_text.value
    sql_wechat_account = "SELECT * FROM case_test.好友聊天记录 WHERE 发送者号码='" + wechat_account + "' OR 接收者号码 = '" + wechat_account + "';"
    #sql_wechat_account = "SELECT * FROM case_test.好友聊天记录 WHERE 发送者号码= %s OR 接收者号码 = %s"
    sql_wechat_string = "SELECT * FROM case_test.好友聊天记录 WHERE 嫌疑人姓名 LIKE '%" + string + "%' OR 内容 LIKE '%" + string + "%' OR 内容翻译 LIKE '%" + string + "%' ;"
    #sql_wechat_string = "SELECT * FROM case_test.好友聊天记录 WHERE 嫌疑人姓名 LIKE '%%s%' OR 内容 LIKE '%%s%' OR 内容翻译 LIKE '%%s%'"
    if wechat_account:
        #print(sql_wechat_account)
        cursor.execute(sql_wechat_account)
        #print(cursor.fetchall())
        df = pd.read_sql(sql_wechat_account, conn)
        display(df)
    if string:
        #print(sql_wechat_string)
        cursor.execute(sql_wechat_string)
        #print(cursor.fetchall())
        df = pd.read_sql(sql_wechat_string, conn)
        display(df)
     
    

In [ ]:
def data_query_geo():
    geo = input_query_geo_text.value
    sql_geo = "SELECT * FROM case_test.应用位置信息 WHERE 地点 LIKE '%" + geo + "%' OR 详细地址 LIKE '%" + geo + "%';"
    #print(sql_geo)
    if geo:
        #print(sql_geo)
        cursor.execute(sql_geo)
        #print(cursor.fetchall())
        df = pd.read_sql(sql_geo, conn)
        display(df)

In [ ]:
from ipywidgets import widgets, Layout  
from IPython.display import display,clear_output

button_clear_out = widgets.Button(description="清除输出")


# Create text widget for input
input_data_import_label = widgets.Label("报告 CSV 文件目录: ")
input_data_import_text = widgets.Text()
input_case_number_label = widgets.Label("案件号: ")
input_case_number_text = widgets.Text()
input_suspect_wechat_account_label = widgets.Label("嫌疑人身份证号: ")
input_suspect_wechat_account_text = widgets.Text()
input_suspect_name_label = widgets.Label("嫌疑人姓名: ")
input_suspect_name_text = widgets.Text()
input_query_wechat_account_label = widgets.Label("搜索微信号: ")
input_query_wechat_account_text = widgets.Text()
input_query_content_string_label = widgets.Label("搜索聊天内容: ")
input_query_content_string_text = widgets.Text()
input_query_case_number_label = widgets.Label("搜索案件号: ")
input_query_case_number_text = widgets.Text()
input_query_geo_label = widgets.Label("碰撞地理位置、wifi等 ")
input_query_geo_text = widgets.Text()
button_import = widgets.Button(description="确定")
button_query = widgets.Button(description="确定")
button_query_geo = widgets.Button(description="确定")

conditions = ["案件录入", "微信聊天内容查询", "应用位置查询"]

tabs_outputs = list()



for current_condition in conditions:
    
    out = widgets.Output(layout={'border': '1px solid black'})
    out = widgets.Output(layout=Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='1px solid green',
                    width='100%',
                    hight='1000%'))
    
    with out:        
        if current_condition == '案件录入':            
            def handle_text_submit(sender):
                on_button_clicked(input_text.value)
            def on_button_clicked(h):
                data_import()            
            display(display(input_data_import_label,input_data_import_text,input_case_number_label,input_case_number_text,input_suspect_wechat_account_label,input_suspect_wechat_account_text,input_suspect_name_label,input_suspect_name_text,button_import,button_clear_out))
            button_import.on_click(on_button_clicked)
            tabs_outputs.append(out)
            #button_clear_out.on_click(clear_output(wait=True))
        if current_condition == '微信聊天内容查询':            
            def handle_text_submit(sender):
                on_button_clicked(input_text.value)
            def on_button_clicked(p):
                data_query_chat()
            display(display(input_query_wechat_account_label,input_query_wechat_account_text,input_query_content_string_label,input_query_content_string_text,input_query_case_number_label,input_query_case_number_text,button_query))
            button_query.on_click(on_button_clicked)
            tabs_outputs.append(out)
        if current_condition == '应用位置查询':
            def handle_text_submit(sender):
                on_button_clicked(input_text.value)
            def on_button_clicked(p):
                data_query_geo()
            display(display(input_query_geo_label,input_query_geo_text,button_query_geo))
            button_query_geo.on_click(on_button_clicked)
            tabs_outputs.append(out)


tab = widgets.Tab()
tab.children = tabs_outputs
for indx in range(len(conditions)):
    tab.set_title(indx, conditions[indx])
tab